In [1]:
import numpy as np
from snnpy import *
import grispy as gsp
from time import time
from tqdm import tqdm 
import gc 

In [2]:

def sigificant_digit(number, digits=2):
    return round(number, digits - int(math.floor(math.log10(abs(number)))) - 1)

def snn_count_neighbors(snn, train, query, radius:list):
    for R in radius:
        return_num = list()
        for j in range(query.shape[0]): 
            ind = snn.query_radius(query[j], R)
            return_num.append(len(ind))
        print("Return: ", sigificant_digit(100*np.mean(return_num)/train.shape[0]), "% neighbors")

### (varying dimensions, fixed size)

In [3]:
dims = [2, 3, 4]
index_sizes = [10000]
query_size = 1000
radius = [0.05, 0.1, 0.15, 0.2, 0.25]

In [4]:
snn_index_time = np.zeros((len(dims), len(index_sizes)))
snn_query_time = np.zeros((len(dims), len(index_sizes)))
gp_index_time = np.zeros((len(dims), len(index_sizes)))
gp_query_time = np.zeros((len(dims), len(index_sizes)))

In [5]:
for i in range(len(dims)):
    print("dim: ", dims[i])
    n_dim = dims[i]
    for j in tqdm(range(len(index_sizes))):
        isize = index_sizes[j]
        data = np.random.uniform(size=(isize + query_size, n_dim))  
        I = data[:isize]
        Q = data[isize:]
        
        inst = time()
        snn_model = build_snn_model(I)  
        snn_index_time[i,j] = time() - inst
        
        inst = time()
        grid = gsp.GriSPy(I)
        gp_index_time[i,j] = time() - inst

        for q in range(query_size):
            qst = time()
            for r in range(len(radius)):
                snnind = snn_model.query_radius(Q[q], radius[r], return_distance=False)
            snn_query_time[i,j] += (time() - qst) / len(radius)
            
            qst = time()
            for r in range(len(radius)):
                gpdist, gpind = grid.bubble_neighbors(Q[[q]], distance_upper_bound=radius[r])
            gp_query_time[i,j] += (time() - qst) / len(radius)
        
        snn_query_time[i,j] = snn_query_time[i,j] / query_size
        gp_query_time[i,j] = gp_query_time[i,j] / query_size
        
        del snn_model, grid
        gc.collect()

dim:  2


100%|██████████| 1/1 [00:19<00:00, 19.41s/it]


dim:  3


100%|██████████| 1/1 [02:06<00:00, 126.91s/it]


dim:  4


100%|██████████| 1/1 [34:12<00:00, 2052.50s/it]


In [6]:
with open('result/add1_snn_index_time.npy', 'wb') as f:
    np.save(f, snn_index_time)
    
with open('result/add1_snn_query_time.npy', 'wb') as f:
    np.save(f, snn_query_time)
    
with open('result/add1_gp_index_time.npy', 'wb') as f:
    np.save(f, gp_index_time)

with open('result/add1_gp_query_time.npy', 'wb') as f:
    np.save(f, gp_query_time)